# DX 704 Week 4 Project

This week's project will test the learning speed of linear contextual bandits compared to unoptimized approaches.
You will start with building a preference data set for evaluation, and then implement different variations of LinUCB and visualize how fast they learn the preferences.


The full project description, a template notebook and supporting code are available on GitHub: [Project 4 Materials](https://github.com/bu-cds-dx704/dx704-project-04).


## Example Code

You may find it helpful to refer to these GitHub repositories of Jupyter notebooks for example code.

* https://github.com/bu-cds-omds/dx601-examples
* https://github.com/bu-cds-omds/dx602-examples
* https://github.com/bu-cds-omds/dx603-examples
* https://github.com/bu-cds-omds/dx704-examples

Any calculations demonstrated in code examples or videos may be found in these notebooks, and you are allowed to copy this example code in your homework answers.

## Part 1: Collect Rating Data

The file "recipes.tsv" in this repository has information about 100 recipes.
Make a new file "ratings.tsv" with two columns, recipe_slug (from recipes.tsv) and rating.
Populate the rating column with values between 0 and 1 where 0 is the worst and 1 is the best.
You can assign these ratings however you want within that range, but try to make it reflect a consistent set of preferences.
These could be your preferences, or a persona of your choosing (e.g. chocolate lover, bacon-obsessed, or sweet tooth).
Make sure that there are at least 10 ratings of zero and at least 10 ratings of one.


Hint: You may find it more convenient to assign raw ratings from 1 to 5 and then remap them as follows.

`ratings["rating"] = (ratings["rating_raw"] - 1) * 0.25`

Submit "ratings.tsv" in Gradescope.

In [5]:
import pandas as pd
import numpy as np


In [2]:
recipes = pd.read_csv("recipes.tsv", sep='\t')
tag = pd.read_csv("recipe-tags.tsv", sep='\t')

In [3]:
recipes.head()

,recipe_slug,recipe_title,recipe_introduction
0,falafel,Falafel,Falafel is a popular Middle Eastern dish made ...
1,spamburger,Spamburger,Spamburger is a type of hamburger that is made...
2,bacon-fried-rice,Bacon Fried Rice,Bacon fried rice is a savory and satisfying di...
3,chicken-fingers,Chicken Fingers,Chicken fingers are a popular dish made from c...
4,apple-crisp,Apple Crisp,Apple crisp is a classic dessert made with bak...


In [20]:
recipes.tail()

,recipe_slug,recipe_title,recipe_introduction
95,bacon-mac-and-cheese,Bacon Mac And Cheese,Bacon mac and cheese is a delicious and comfor...
96,chicken-alfredo-lasagna,Chicken Alfredo Lasagna,Chicken alfredo lasagna is a delicious twist o...
97,classic-beef-lasagna,Classic Beef Lasagna,Classic beef lasagna is a hearty and comfortin...
98,vegetarian-mushroom-lasagna,Vegetarian Mushroom Lasagna,Vegetarian mushroom lasagna is a delicious and...
99,spinach-and-ricotta-lasagna,Spinach And Ricotta Lasagna,Spinach and ricotta lasagna is a delicious veg...


In [4]:
tag.head()

,recipe_slug,recipe_tag
0,spam-musubi,hawaiian
1,spam-musubi,nori
2,spam-musubi,onthego
3,spam-musubi,rice
4,spam-musubi,snack


In [40]:
# prefernces
love = ["peanut-butter","hamburger","burger","beef","chicken"]
hate = ["seafood","fish","shrimp","salmon","tuna","crab","lobster", "sushi","capers","kale","mushroom","jalapenos","spam"]

In [19]:
love_slug = set(
    tag[tag["recipe_tag"].str.contains("|".join(love), case=False, na=False)]["recipe_slug"])

In [41]:
hate_slug = set(
    tag[tag["recipe_tag"].str.contains("|".join(hate), case=False, na=False)]["recipe_slug"])

In [37]:
ratings = recipes[["recipe_slug"]].copy()
ratings["rating"] = 0.5  # default neutral rating

In [42]:
ratings.loc[ratings["recipe_slug"].isin(love_slug), "rating"] = 1.0


ratings.loc[ratings["recipe_slug"].isin(hate_slug), "rating"] = 0.0

In [45]:
# check 10 
print("Number of 1.0 ratings:", (ratings["rating"] == 1.0).sum())
print("Number of 0.0 ratings:", (ratings["rating"] == 0.0).sum())

Number of 1.0 ratings: 14
Number of 0.0 ratings: 10


In [44]:
ratings.to_csv("ratings.tsv", sep="\t", index=False)

print("ratings.tsv file created successfully.")

ratings.tsv file created successfully.


## Part 2: Construct Model Input

Use your file "ratings.tsv" combined with "recipe-tags.tsv" to create a new file "features.tsv" with a column recipe_slug, a column bias which is hard-coded to one, and a column for each tag that appears in "recipe-tags.tsv".
The tag column in this file should be a 0-1 encoding of the recipe tags for each recipe.
[Pandas reshaping function methods](https://pandas.pydata.org/docs/user_guide/reshaping.html) may be helpful.

The bias column will make later LinUCB calculations easier since it will just be another dimension.

Hint: For later modeling steps, it will be important to have the feature data (inputs) and the rating data (target outputs) in the same order.
It is highly recommended to make sure that "features.tsv" and "ratings.tsv" have the recipe slugs in the same order.

In [51]:
# YOUR CHANGES HERE

tag = tag.assign(value=1)

tag_matrix = tag.pivot_table(
    index="recipe_slug",
    columns="recipe_tag",
    values="value",
    aggfunc="max",     
    fill_value=0
).astype(int)


slugs_order = ratings["recipe_slug"]
tag_matrix_ordered = tag_matrix.reindex(slugs_order, fill_value=0)



In [52]:
features = pd.DataFrame({"recipe_slug": slugs_in_order})
features["bias"] = 1
features = pd.concat([features, tag_matrix_ordered.reset_index(drop=True)], axis=1)


In [54]:
features.to_csv("features.tsv", sep="\t", index=False)

#check
print("features.tsv created.")
print("Shape:", features.shape)
print("Preview columns:", features.columns[:10].tolist())

features.tsv created.
Shape: (100, 298)
Preview columns: ['recipe_slug', 'bias', 'alfredo', 'almond', 'american', 'appetizer', 'appetizers', 'apple', 'asiancuisine', 'asparagus']


Submit "features.tsv" in Gradescope.

## Part 3: Linear Preference Model

Use your feature and rating files to build a ridge regression model with ridge regression's regularization parameter $\alpha$ set to 1.


Hint: If you are using scikit-learn modeling classes, you should use `fit_intercept=False` since that intercept value will be redundant with the bias coefficient.

Hint: The estimate component of the bounds should match the previous estimate, so you should be able to just focus on the variance component of the bounds now.

In [58]:
# YOUR CHANGES HERE

from sklearn.linear_model import Ridge

assert all(features["recipe_slug"] == ratings["recipe_slug"])

X = features.drop(columns=["recipe_slug"])
y = ratings["rating"]


model = Ridge(alpha=1, fit_intercept=False)
model.fit(X, y)


coefficients = pd.DataFrame({
    "recipe_tag": X.columns,
    "coefficient": model.coef_
})



Save the coefficients of this model in a file "model.tsv" with columns "recipe_tag" and "coefficient".
Do not add anything for the `intercept_` attribute of a scikit-learn model; this will be covered by the coefficient for the bias column added in part 2.

In [59]:
# YOUR CHANGES HERE

coefficients.to_csv("model.tsv", sep="\t", index=False)

Submit "model.tsv" in Gradescope.

## Part 4: Recipe Estimates

Use the recipe model to estimate the score of every recipe.
Save these estimates to a file "estimates.tsv" with columns recipe_slug and score_estimate.

In [60]:
# YOUR CHANGES HERE

score_recipe = model.predict(X)


estimates = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_estimate": score_recipe
})




In [61]:

estimates.to_csv("estimates.tsv", sep="\t", index=False)

Submit "estimates.tsv" in Gradescope.

## Part 5: LinUCB Bounds

Calculate the upper bounds of LinUCB using data corresponding to trying every recipe once and receiving the rating in "ratings.tsv" as the reward.
Keep the ridge regression regularization parameter at 1, and set LinUCB's $\alpha$ parameter to 2.
Save these upper bounds to a file "bounds.tsv" with columns recipe_slug and score_bound.

In [ ]:
# YOUR CHANGES HERE

features = pd.read_csv("features.tsv", sep="\t")   
ratings  = pd.read_csv("ratings.tsv", sep="\t")    

# same order
assert (features["recipe_slug"].values == ratings["recipe_slug"].values).all(), \
    "features.tsv and ratings.tsv are not in the same recipe_slug order."

# prep x y 
X = features.drop(columns=["recipe_slug"]).to_numpy(dtype=float)  # (n, d)
y = ratings["rating"].to_numpy(dtype=float)                       # (n,)

n, d = X.shape


# lin ridge config
lam = 1.0      # ridge regularization parameter (lambda)
alpha = 2.0    # LinUCB exploration parameter

# compute
A = X.T @ X + lam * np.eye(d)
b = X.T @ y

# solve
theta_hat = np.linalg.solve(A, b)

# ucb var
A_inv = np.linalg.inv(A)  

# upper
mean_est = X @ theta_hat  # (n,)


AinvX_T = (A_inv @ X.T).T          # (n, d) each row is (A_inv x_i)
var = np.sum(X * AinvX_T, axis=1)  # (n,)
ucb = mean_est + alpha * np.sqrt(var)



In [63]:
bounds = pd.DataFrame({
    "recipe_slug": features["recipe_slug"],
    "score_bound": ucb
})
bounds.to_csv("bounds.tsv", sep="\t", index=False)


Submit "bounds.tsv" in Gradescope.

## Part 6: Make Online Recommendations

Implement LinUCB to make 100 recommendations starting with no data and using the same parameters as in part 5.
One recommendation should be made at a time and you can break ties arbitrarily.
After each recommendation, use the rating from part 1 as the reward to update the LinUCB data.
Record the recommendations made in a file "recommendations.tsv" with columns "recipe_slug", "score_bound", and "reward".
The rows in this file should be in the same order as the recommendations were made.

Hint: do not remove recipes after each recommendation.
Repeating recommendations is expected.

In [64]:
# YOUR CHANGES HERE


# X matrix reward 
X = features.drop(columns=["recipe_slug"]).to_numpy(dtype=float)  
slugs = features["recipe_slug"].tolist()
reward_map = dict(zip(ratings["recipe_slug"], ratings["rating"]))

n, d = X.shape

# linucb para
lam = 1.0     
alpha = 2.0   
T = 100       

# initalize no data
A = lam * np.eye(d)
b = np.zeros(d)

rows = []

# run linucb
for t in range(T):
   
    theta_hat = np.linalg.solve(A, b)

    # score and solve
    Ainv_XT = np.linalg.solve(A, X.T).T           
    mean_est = X @ theta_hat                      
    var = np.sum(X * Ainv_XT, axis=1)             
    score_bound = mean_est + alpha * np.sqrt(var) 

    # pick best
    i = int(np.argmax(score_bound))
    chosen_slug = slugs[i]
    chosen_x = X[i]

    # reward
    r = float(reward_map[chosen_slug])

    # record
    rows.append({
        "recipe_slug": chosen_slug,
        "score_bound": float(score_bound[i]),
        "reward": r
    })

    # update
    A += np.outer(chosen_x, chosen_x)
    b += r * chosen_x



In [65]:
recs = pd.DataFrame(rows, columns=["recipe_slug", "score_bound", "reward"])
recs.to_csv("recommendations.tsv", sep="\t", index=False)

Submit "recommendations.tsv" in Gradescope.

## Part 7: Acknowledgments

Make a file "acknowledgments.txt" documenting any outside sources or help on this project.
If you discussed this assignment with anyone, please acknowledge them here.
If you used any libraries not mentioned in this module's content, please list them with a brief explanation what you used them for.
If you used any generative AI tools, please add links to your transcripts below, and any other information that you feel is necessary to comply with the generative AI policy.
If no acknowledgements are appropriate, just write none in the file.


Submit "acknowledgments.txt" in Gradescope.

## Part 8: Code

Please submit a Jupyter notebook that can reproduce all your calculations and recreate the previously submitted files.


Submit "project.ipynb" in Gradescope.